<a href="https://colab.research.google.com/github/rachitt-t/AI-in-healthcare/blob/main/lab_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RACHIT TAYAL

In [3]:
# pima_clustering.py
# Complete pipeline: EDA, preprocessing, KMeans + Hierarchical clustering, visualization.
# Requirements: pandas, numpy, matplotlib, scikit-learn, scipy, seaborn (optional, but not used for plots here)
# Save as pima_clustering.py or run in a Jupyter notebook cell.

import os
import urllib.request
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.pipeline import Pipeline

# -----------------------
# 1) Download / load data
# -----------------------
DATA_URL = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
# Alternate: https://archive.ics.uci.edu/ml/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.data
# Columns per UCI / Kaggle:
COLUMNS = ["Pregnancies", "Glucose", "BloodPressure", "SkinThickness",
           "Insulin", "BMI", "DiabetesPedigreeFunction", "Age", "Outcome"]

outdir = "pima_outputs"
os.makedirs(outdir, exist_ok=True)

# try to download (works if internet available); if already downloaded, you can load locally.
local_csv = os.path.join(outdir, "pima-indians-diabetes.csv")
if not os.path.exists(local_csv):
    print("Downloading dataset...")
    urllib.request.urlretrieve(DATA_URL, local_csv)
else:
    print("Using cached dataset at", local_csv)

df = pd.read_csv(local_csv, header=None, names=COLUMNS)
print("Loaded dataset shape:", df.shape)

# -----------------------
# Task 1: EDA
# -----------------------
# 1. Summary statistics
summary = df.describe().T
summary['median'] = df.median()
print("\nSummary statistics (first rows):\n", summary.head(10))

# 2. Missing values, duplicates, outliers check
print("\nMissing values (count per column):")
print(df.isnull().sum())

print("\nDuplicates:", df.duplicated().sum())

# Note: In this dataset several features use zeros to encode missing values:
# Glucose, BloodPressure, SkinThickness, Insulin, BMI — zeros are physiologically implausible.
zero_missing_cols = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
for col in zero_missing_cols:
    n_zero = (df[col] == 0).sum()
    print(f"Zeros in {col}: {n_zero}")

# Save the raw histograms / boxplots / correlation heatmap
# Use matplotlib only (no seaborn) per instructions
# Histograms
plt.figure(figsize=(12, 8))
df.hist(bins=25, figsize=(12,8))
plt.tight_layout()
plt.suptitle("Feature histograms (raw)", y=1.02)
plt.savefig(os.path.join(outdir, "histograms_raw.png"), bbox_inches='tight')
plt.close()

# Boxplots for a subset
plt.figure(figsize=(12,6))
for idx, col in enumerate(df.columns[:-1], 1):
    plt.subplot(2,4,idx)
    plt.boxplot(df[col].dropna(), vert=True, showfliers=True)
    plt.title(col)
plt.tight_layout()
plt.savefig(os.path.join(outdir, "boxplots_raw.png"))
plt.close()

# Correlation heatmap (matshow)
corr = df.corr()
fig, ax = plt.subplots(figsize=(8,6)) # Create figure and axes explicitly
im = ax.matshow(corr) # Use the axes for plotting
ax.set_xticks(range(len(corr.columns)), corr.columns, rotation=90)
ax.set_yticks(range(len(corr.columns)), corr.columns)
fig.colorbar(im, ax=ax) # Use the axes for the colorbar
ax.set_title("Correlation matrix (raw) - colorbar on right", pad=20)
plt.savefig(os.path.join(outdir, "correlation_raw.png"), bbox_inches='tight')
plt.close(fig) # Close the explicitly created figure

# Class distribution
counts = df["Outcome"].value_counts()
print("\nClass distribution (Outcome):")
print(counts)
plt.figure(figsize=(4,3))
plt.bar(counts.index.astype(str), counts.values)
plt.xlabel("Outcome")
plt.ylabel("Count")
plt.title("Class distribution (Outcome)")
plt.savefig(os.path.join(outdir, "class_distribution.png"))
plt.close()

# -----------------------
# Task 2: Preprocessing (for clustering we drop Outcome)
# -----------------------
X = df.drop(columns=["Outcome"]).copy()

# Treat zeros in specific columns as missing (NaN) and impute (median)
X[zero_missing_cols] = X[zero_missing_cols].replace(0, np.nan)

print("\nAfter replacing zeros with NaN in selected cols:")
print(X[zero_missing_cols].isnull().sum())

# Impute with median
imputer = SimpleImputer(strategy="median")
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Standardize
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X_imputed.columns)

# Save preprocessed sample
X_scaled.head().to_csv(os.path.join(outdir, "X_scaled_head.csv"), index=False)

# -----------------------
# Task 3: K-means clustering
# -----------------------
# 3a) Determine optimal K using elbow (inertia) and silhouette
inertia = []
sil_scores = []
K_range = range(2, 11)
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    inertia.append(kmeans.inertia_)
    sil_scores.append(silhouette_score(X_scaled, labels))

# Plot elbow
plt.figure()
plt.plot(list(K_range), inertia, marker='o')
plt.xlabel("K")
plt.ylabel("Inertia")
plt.title("Elbow method (inertia) for K-means")
plt.savefig(os.path.join(outdir, "kmeans_elbow.png"))
plt.close()

# Plot silhouette vs K
plt.figure()
plt.plot(list(K_range), sil_scores, marker='o')
plt.xlabel("K")
plt.ylabel("Silhouette score")
plt.title("Silhouette score for different K")
plt.savefig(os.path.join(outdir, "kmeans_silhouette.png"))
plt.close()

# Choose K with highest silhouette or elbow. We'll pick K = argmax(silhouette) as a heuristic:
best_k = K_range[int(np.argmax(sil_scores))]
print(f"\nBest K by silhouette: {best_k} (silhouette={max(sil_scores):.4f})")

kmeans_best = KMeans(n_clusters=best_k, random_state=42, n_init=20)
kmeans_labels = kmeans_best.fit_predict(X_scaled)

# -----------------------
# Visualization: PCA 2D colored by KMeans clusters
# -----------------------
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(8,6))
for lab in np.unique(kmeans_labels):
    plt.scatter(X_pca[kmeans_labels==lab, 0], X_pca[kmeans_labels==lab, 1], label=f"Cluster {lab}", alpha=0.6)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title(f"KMeans clusters (K={best_k}) on first 2 PCA components")
plt.legend()
plt.savefig(os.path.join(outdir, "kmeans_pca.png"))
plt.close()

# -----------------------
# Agglomerative hierarchical clustering
# -----------------------
# Try different linkage methods and plot dendrogram for a sample (dendrogram cost grows quickly)
sample_size = 200
np.random.seed(42)
sample_idx = np.random.choice(len(X_scaled), size=sample_size, replace=False)
X_sample = X_scaled.iloc[sample_idx]

# compute linkage matrix for 'ward' and 'average' and 'complete'
linkages = ["ward", "average", "complete"]
for link in linkages:
    Z = linkage(X_sample, method=link)
    plt.figure(figsize=(12, 5))
    dendrogram(Z, truncate_mode='level', p=12)
    plt.title(f"Dendrogram (linkage={link}) - truncated")
    plt.xlabel("sample index (truncated)")
    plt.ylabel("distance")
    plt.savefig(os.path.join(outdir, f"dendrogram_{link}.png"))
    plt.close()

# Choose number of clusters from dendrogram subjectively; use same best_k for comparison
agg = AgglomerativeClustering(n_clusters=best_k, linkage='ward')  # ward is common when using Euclidean distance
agg_labels = agg.fit_predict(X_scaled)

# PCA visualization for hierarchical clusters
plt.figure(figsize=(8,6))
for lab in np.unique(agg_labels):
    plt.scatter(X_pca[agg_labels==lab, 0], X_pca[agg_labels==lab, 1], label=f"Cluster {lab}", alpha=0.6)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title(f"Agglomerative Clustering (n_clusters={best_k}, linkage=ward) on PCA(2)")
plt.legend()
plt.savefig(os.path.join(outdir, "agg_pca.png"))
plt.close()

# -----------------------
# Comparison and external evaluation with Outcome (optional)
# -----------------------
# Show contingency of cluster assignments vs actual Outcome to get an intuition (external eval)
cont_kmeans = pd.crosstab(kmeans_labels, df["Outcome"])
cont_agg = pd.crosstab(agg_labels, df["Outcome"])

print("\nContingency table: KMeans clusters vs Outcome")
print(cont_kmeans)
print("\nContingency table: Agglomerative clusters vs Outcome")
print(cont_agg)

# Silhouette of final chosen clustering
sil_kmeans = silhouette_score(X_scaled, kmeans_labels)
sil_agg = silhouette_score(X_scaled, agg_labels)
print(f"\nSilhouette (KMeans, K={best_k}): {sil_kmeans:.4f}")
print(f"Silhouette (Agglomerative, n_clusters={best_k}): {sil_agg:.4f}")

# -----------------------
# Save key outputs (silhouette, chosen K)
# -----------------------
with open(os.path.join(outdir, "summary.txt"), "w") as f:
    f.write("Pima clustering summary\n")
    f.write(f"Best K by silhouette: {best_k}\n")
    f.write(f"Silhouette KMeans: {sil_kmeans:.4f}\n")
    f.write(f"Silhouette Agglomerative: {sil_agg:.4f}\n")
    f.write("\nKMeans cluster counts:\n")
    f.write(str(pd.Series(kmeans_labels).value_counts().to_dict()) + "\n")
    f.write("\nAgglomerative cluster counts:\n")
    f.write(str(pd.Series(agg_labels).value_counts().to_dict()) + "\n")

print(f"\nAll plots and outputs saved to folder: {outdir}")

Using cached dataset at pima_outputs/pima-indians-diabetes.csv
Loaded dataset shape: (768, 9)

Summary statistics (first rows):
                           count        mean         std     min       25%  \
Pregnancies               768.0    3.845052    3.369578   0.000   1.00000   
Glucose                   768.0  120.894531   31.972618   0.000  99.00000   
BloodPressure             768.0   69.105469   19.355807   0.000  62.00000   
SkinThickness             768.0   20.536458   15.952218   0.000   0.00000   
Insulin                   768.0   79.799479  115.244002   0.000   0.00000   
BMI                       768.0   31.992578    7.884160   0.000  27.30000   
DiabetesPedigreeFunction  768.0    0.471876    0.331329   0.078   0.24375   
Age                       768.0   33.240885   11.760232  21.000  24.00000   
Outcome                   768.0    0.348958    0.476951   0.000   0.00000   

                               50%        75%     max    median  
Pregnancies                 3.0000

<Figure size 1200x800 with 0 Axes>